In [ ]:
import torch
import os
from PIL import Image
from pathlib import Path
import pandas as pd
import torchvision
from sklearn.model_selection import train_test_split

In [ ]:
labels=pd.read_csv('/kaggle/input/dog-breed-identification/labels.csv')
train,valid=train_test_split(labels,train_size=0.8,shuffle=True,stratify=labels['breed'],random_state=42)

In [ ]:
train,train_labels=train['id'].reset_index(drop=True),train['breed'].reset_index(drop=True)
val,val_labels=valid['id'].reset_index(drop=True),valid['breed'].reset_index(drop=True)

In [ ]:
'''Encoding labels'''

breeds=dict()
breed_count=1

for breed in labels['breed'].value_counts().index:

    breeds[breed]=breed_count-1
    breed_count+=1

val_labels_torch=torch.zeros(len(val_labels),1)
train_labels_torch=torch.zeros(len(train_labels),1)

for index in val_labels.index:
    val_labels_torch[index]=breeds[val_labels.iloc[index]]

for index in train_labels.index:
    train_labels_torch[index]=breeds[train_labels.iloc[index]]

val_labels_torch = val_labels_torch.long()
train_labels_torch = train_labels_torch.long()


val_labels_torch[91]
# val_labels=torch.tensor(val_labels.values)
# train_labels=torch.tensor(train_labels.values)

In [ ]:
transform = torchvision.transforms.Compose([
    torchvision.transforms.Resize((224, 224)),
    torchvision.transforms.RandomHorizontalFlip(),
    torchvision.transforms.RandomRotation(10),
    torchvision.transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2),
    torchvision.transforms.ToTensor()
])


def create_dataset(series, directory_path, transform):
    tensors = []
    for i in range(len(series)):
        if(series[i].endswith('.jpg')):
            img_file=series[i]
        else:
            img_file = series[i] + '.jpg'
        img_path = os.path.join(directory_path, img_file)
        img = Image.open(img_path).convert('RGB')  # Ensure RGB
        img_t = transform(img)
        tensors.append(img_t)
    return torch.stack(tensors, dim=0)


In [ ]:
directory_path=Path('/kaggle/input/dog-breed-identification/train')
train_dataset=create_dataset(train,directory_path,transform)

In [ ]:


# For validation: no augmentation
val_transform = torchvision.transforms.Compose([
    torchvision.transforms.Resize((224, 224)),
    torchvision.transforms.ToTensor()
])
val_dataset = create_dataset(val, directory_path, transform=val_transform)


In [ ]:
from torch.utils.data import TensorDataset

train_dataset_torch=TensorDataset(train_dataset,train_labels_torch)
val_dataset_torch=TensorDataset(val_dataset,val_labels_torch)

In [ ]:
train_dataset.shape,train_labels_torch.shape

In [ ]:
val_dataset_torch[0]

In [ ]:
# ========================
# 1. 環境設定與依賴導入
# ========================
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms, models
from torch.cuda.amp import GradScaler, autocast
import pandas as pd
import numpy as np
import os
from PIL import Image
from sklearn.model_selection import train_test_split

# 自動檢測設備
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"使用設備: {device}")

# ========================
# 2. 資料增強與預處理
# ========================
train_transform = transforms.Compose([
    transforms.Resize((600, 600)),  # EfficientNet-B7 標準輸入尺寸
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(15),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

val_transform = transforms.Compose([
    transforms.Resize((600, 600)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# ========================
# 3. 自訂資料集類別
# ========================
class DogBreedDataset(Dataset):
    def __init__(self, dataframe, img_dir, transform=None):
        self.df = dataframe
        self.img_dir = img_dir
        self.transform = transform
        
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, idx):
        img_name = self.df.iloc[idx]['id'] + '.jpg'
        label = self.df.iloc[idx]['breed']
        img_path = os.path.join(self.img_dir, img_name)
        image = Image.open(img_path).convert('RGB')
        
        if self.transform:
            image = self.transform(image)
            
        return image, label

# ========================
# 4. 載入資料集
# ========================
labels = pd.read_csv('/kaggle/input/dog-breed-identification/labels.csv')
train_df, val_df = train_test_split(labels, test_size=0.2, stratify=labels['breed'], random_state=42)

train_dataset = DogBreedDataset(train_df, '/kaggle/input/dog-breed-identification/train', train_transform)
val_dataset = DogBreedDataset(val_df, '/kaggle/input/dog-breed-identification/train', val_transform)

# 建立資料加載器
batch_size = 16  # 根據 GPU 記憶體調整
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=4, pin_memory=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=4, pin_memory=True)

# ========================
# 5. 模型定義（含權重自動下載）
# ========================
def load_efficientnet_b7():
    try:
        # 嘗試自動下載權重
        model = models.efficientnet_b7(weights=models.EfficientNet_B7_Weights.IMAGENET1K_V1)
        print("✅ 成功載入官方預訓練權重")
    except Exception as e:
        print(f"⚠️ 自動下載失敗: {str(e)}")
        print("請手動下載權重並上傳到 /kaggle/input/efficientnet-b7-weights/")
        try:
            # 載入本地權重
            model = models.efficientnet_b7(weights=None)
            state_dict = torch.load('/kaggle/input/efficientnet-b7-weights/efficientnet_b7.pth')
            model.load_state_dict(state_dict)
            print("✅ 成功載入本地預訓練權重")
        except:
            print("❌ 無法載入任何權重，使用隨機初始化")
            model = models.efficientnet_b7(weights=None)
    
    # 凍結特徵提取層
    for param in model.features.parameters():
        param.requires_grad = False
    
    # 修改分類頭
    model.classifier = nn.Sequential(
        nn.Dropout(0.5),
        nn.Linear(2560, 1024),
        nn.SiLU(),
        nn.BatchNorm1d(1024),
        nn.Dropout(0.3),
        nn.Linear(1024, 512),
        nn.SiLU(),
        nn.Linear(512, 120)
    )
    return model.to(device)

model = load_efficientnet_b7()

# ========================
# 6. 訓練參數設定
# ========================
criterion = nn.CrossEntropyLoss()
optimizer = optim.AdamW(model.parameters(), lr=1e-4, weight_decay=1e-4)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'max', patience=3, factor=0.5)
scaler = GradScaler()  # 混合精度訓練
best_accuracy = 0.0

# ========================
# 7. 訓練循環（含混合精度）
# ========================
for epoch in range(50):  # 總訓練輪數
    # 訓練階段
    model.train()
    train_loss = 0.0
    for images, labels in train_loader:
        images = images.to(device, non_blocking=True)
        labels = labels.to(device, non_blocking=True)
        
        optimizer.zero_grad()
        
        with autocast():
            outputs = model(images)
            loss = criterion(outputs, labels)
        
        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()
        
        train_loss += loss.item() * images.size(0)
    
    # 驗證階段
    model.eval()
    val_loss = 0.0
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in val_loader:
            images = images.to(device, non_blocking=True)
            labels = labels.to(device, non_blocking=True)
            
            outputs = model(images)
            loss = criterion(outputs, labels)
            val_loss += loss.item() * images.size(0)
            
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    
    # 計算指標
    train_loss = train_loss / len(train_loader.dataset)
    val_loss = val_loss / len(val_loader.dataset)
    accuracy = 100 * correct / total
    
    # 學習率調整
    scheduler.step(accuracy)
    
    # 保存最佳模型
    if accuracy > best_accuracy:
        best_accuracy = accuracy
        torch.save(model.state_dict(), 'best_model.pth')
        print(f"🎉 發現新最佳模型，準確率: {accuracy:.2f}%")
    
    print(f"Epoch {epoch+1}/50 | "
          f"Train Loss: {train_loss:.4f} | "
          f"Val Loss: {val_loss:.4f} | "
          f"Accuracy: {accuracy:.2f}% | "
          f"LR: {optimizer.param_groups[0]['lr']:.2e}")

print(f"訓練完成！最佳驗證準確率: {best_accuracy:.2f}%")


In [ ]:
from torch.cuda.amp import autocast, GradScaler
from torch.optim.lr_scheduler import ReduceLROnPlateau

# 初始化
scaler = GradScaler()
scheduler = ReduceLROnPlateau(optimizer, 'min', patience=3, factor=0.5)
best_loss = float('inf')
patience = 5
no_improve = 0

# 數據加載優化
train_loader = DataLoader(
    train_dataset_torch,
    batch_size=128,
    shuffle=True,
    num_workers=4,
    pin_memory=True
)

for i in range(1, n_epochs+1):
    model_torch.train()
    training_loss = 0.0
    for img, label in train_loader:
        img = img.to('cuda', non_blocking=True)
        label = label.to('cuda', non_blocking=True).squeeze(1)
        
        optimizer.zero_grad()
        with autocast():
            outputs = model_torch(img)
            losses = loss_fn(outputs, label)
        scaler.scale(losses).backward()
        scaler.step(optimizer)
        scaler.update()
        training_loss += losses.item()
    
    avg_train_loss = training_loss / len(train_loader)
    
    # 驗證階段
    model_torch.eval()
    val_loss = 0.0
    correct = 0
    total = 0
    with torch.no_grad():
        for img, label in val_loader:
            img = img.to('cuda', non_blocking=True)
            label = label.to('cuda', non_blocking=True).squeeze(1)
            outputs = model_torch(img)
            loss = loss_fn(outputs, label)
            val_loss += loss.item()
            _, predicted = torch.max(outputs, 1)
            total += label.size(0)
            correct += (predicted == label).sum().item()
    
    avg_val_loss = val_loss / len(val_loader)
    accuracy = 100 * correct / total
    
    # 學習率調整與早停
    scheduler.step(avg_val_loss)
    if avg_val_loss < best_loss:
        best_loss = avg_val_loss
        no_improve = 0
        torch.save(model_torch.state_dict(), 'best_model.pth')
    else:
        no_improve +=1
    
    if no_improve >= patience:
        print(f"Early stopping at epoch {i}!")
        break
    
    print(f"Epoch [{i}/{n_epochs}], Train Loss: {avg_train_loss:.4f}, Val Loss: {avg_val_loss:.4f}, Acc: {accuracy:.2f}%")


In [ ]:
# 绘制损失和准确率曲线
plt.figure(figsize=(12, 5))

# 子图1：损失曲线
plt.subplot(1, 2, 1)
plt.plot(range(1, n_epochs+1), train_losses, 'b-', label='Training Loss')
plt.plot(range(1, n_epochs+1), val_losses, 'r-', label='Validation Loss')
plt.title('Loss Curve')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.grid(True, linestyle='--', alpha=0.5)
plt.legend()

# 子图2：准确率曲线
plt.subplot(1, 2, 2)
plt.plot(range(1, n_epochs+1), accuracies, 'g-', label='Accuracy')
plt.title('Validation Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy (%)')
plt.grid(True, linestyle='--', alpha=0.5)
plt.legend()

plt.tight_layout()
plt.savefig('training_curves.png')  # 保存图片
plt.show()

In [ ]:
torch.save(resnet.state_dict(), 'resnet_custom_head.pth')


In [ ]:
os.listdir('/kaggle/input/dog-breed-identification/test')[1:10]

In [ ]:

test=os.listdir('/kaggle/input/dog-breed-identification/test')
test_dataset=create_dataset(test,'/kaggle/input/dog-breed-identification/test',val_transform)
test_dataset=test_dataset.to(device='cuda')

In [ ]:
import pandas as pd
output=pd.DataFrame(columns=['id',*breeds.keys()])

In [ ]:
output

In [ ]:
breeds

In [ ]:
import torch.nn.functional as F
i=0
for file,sample in zip(os.listdir('/kaggle/input/dog-breed-identification/test'),test_dataset):
    sample=sample.cuda()
    output_values=model_torch(sample.unsqueeze(dim=0))
    output_values=output_values.cpu()
    probabilities = F.softmax(output_values, dim=1)
    file_id = file.split('.')[0]  # 移除 .jpg 副檔名
    output.loc[i] = [file_id, *probabilities[0].detach().numpy()] 
    i+=1

In [ ]:
output.head()

In [ ]:
output.to_csv('test_submission.csv',index=False)